<a href="https://colab.research.google.com/github/Fazira2025/Fazira-Harahap_Weather-Wise_FINAL/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---



In [39]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install requests
!pip install matplotlib
!pip install pyinputplus

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyinputplus: filename=pyinputplus-0.2.12-py3-none-any.whl size=11348 sha256=9322cdc3e3b77ef9e9c369a12bb65876ec467589c9296c456158d319824a32ad
  Stored in directory: /root/.cache/pip/wheels/a3/6b/89/3bec14288af8e0729d088fb41a785e583d49e772f2529c7676
  Created wheel for pysimplevalidate: filename=pysimplevalidate-0.2.12-py3-none-any.whl size=16226 sha256=712123db632ebe015dd70a0304fda40d77c4eb7ca0be515b77baa3e79fa0d673
  Stored in directory: /root/.cache/pip/wheels/fc/40/7a/4d3b2dc2e80b4b3bbda89eec94fcefd6c8bd1101cc7bb89554
  Created wheel for stdiomask: filename=stdiomask

## 📦 Setup and Configuration
Import required packages and setup environment.

In [40]:
# Add any other setup code here
import requests #call API
import matplotlib.pyplot as plt #call graphic
import re #call NLP
from datetime import datetime, timedelta #call datetime

## 🌤️ Weather Data Functions

In [41]:
# Define get_weather_data here - to call API
def get_weather_data(location, format_type='plaintext'):
    """
    Get weather data from wttr.in API
    Args:
        location (str): City name or location
    Returns:
        str: Weather data in text format or None if an error occurs
    Raises:
        Various requests exceptions which are caught and printed
    """
    try:
        if format_type.lower() == "json": #call JSON format
            url = f'https://wttr.in/{location}?format=j1'
        else:
            url = f'https://wttr.in/{location}'

        response = requests.get(url, timeout=10) #link to API with time limit 10 sec
        response.raise_for_status()

        if format_type.lower() == "json":
            return response.json()
        else:
            print(response.text)
            return None
    #error handling http request
    except requests.exceptions.HTTPError as errh:
        print(f"HTTP Error: {errh}")
    except requests.exceptions.ConnectionError as errc:
        print(f"Connection Error: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Timeout Error: {errt}")
    except requests.exceptions.TooManyRedirects as errr:
        print(f"Redirect Error: {errr}")
    except requests.exceptions.RequestException as err:
        print(f"Unknown Error: {err}")

    return None

## 📊 Visualisation Functions

In [34]:
# Define specific keywords for each weather condition #list_dictionary - for visualization menu 3
def create_temperature_visualisation(location, target_date):
    data = get_weather_data(location, format_type="json") #parsing with JSON #calling API get_weather_data
    if data is None:
        print("Failed to retrieve weather data.")
        return None

    try:
        weather_days = data['weather']  # List of daily forecasts
    except KeyError:
        print("Unexpected data format.")
        return None

    target_date_str = target_date.strftime("%Y-%m-%d")

    for day in weather_days:
        if day.get('date') == target_date_str:
            date = day.get('date', 'Unknown Date')
            try:
                dt_obj = datetime.strptime(date, "%Y-%m-%d")
                formatted_date = dt_obj.strftime("%A, %d %B %Y")
            except Exception:
                formatted_date = date

            weather_desc = day.get('hourly', [{}])[4].get('weatherDesc', [{'value': 'N/A'}])[0]['value'] #result of JSON
            mintempC = day.get('mintempC', 'N/A')
            maxtempC = day.get('maxtempC', 'N/A')
            avgtempC = day.get('avgtempC', 'N/A')
            weather_emoji = get_emoji(weather_desc)

            # Return data dictionary instead of printing - weather_desc consists of the return value (location/date/emojis/min/max)
            return {
                "location": location.capitalize(),
                "date": formatted_date,
                "weather_desc": weather_desc,
                "weather_emoji": weather_emoji,
                "min_temp": mintempC,
                "max_temp": maxtempC,
                "avg_temp": avgtempC
            }
    print(f"No weather data found for {target_date_str}.")
    return None

#----------------------------------------------------------------------
def get_emoji(weather_desc): #use to visualize weather_desc function wttr API
    """
    Return an emoji corresponding to the weather description.()
    """
    weather_lower = weather_desc.lower()

    rain_keywords = ['rain', 'drizzle', 'shower', 'thundery outbreaks', 'downpour', 'wet']
    snow_keywords = ['snow', 'blizzard', 'sleet', 'ice', 'hail', 'flurry']
    cloud_keywords = ['cloud', 'overcast', 'mist', 'fog', 'haze', 'partly cloudy', 'scattered clouds']
    fog_keywords = ['mist', 'fog', 'haze']
    sun_keywords = ['sunny', 'clear', 'fair', 'hot', 'bright', 'clear skies', 'clear weather']
    wind_keywords = ['wind', 'gale', 'storm', 'breeze', 'gust']

#if function if any keywords in the dictionary called
 # Rain conditions
    if any(word in weather_lower for word in rain_keywords):
        if 'thunder' in weather_lower or 'storm' in weather_lower or 'thundery outbreaks' in weather_lower:
            return '⛈️'  # Thunderstorm
        elif 'light' in weather_lower or 'patchy' in weather_lower:
            return '🌦️'  # Sun behind rain cloud
        else:
            return '🌧️'  # Rain cloud

    # Snow conditions
    elif any(word in weather_lower for word in snow_keywords):
        if 'light' in weather_lower or 'patchy' in weather_lower:
            return '🌨️'  # Snow cloud
        else:
            return '❄️'  # Snowflake

    # Cloud conditions
    elif any(word in weather_lower for word in cloud_keywords):
        if 'fog' in weather_lower or 'mist' in weather_lower:
            return '🌫️'  # Fog
        elif 'overcast' in weather_lower:
            return '☁️'  # Cloudy weather
        elif 'partly' in weather_lower or 'scattered' in weather_lower:
            return '⛅'  # Sun behind cloud
        else:
            return '☁️'  # Cloud

    # Clear conditions (sunny/fair weather)
    elif any(word in weather_lower for word in sun_keywords):
        return '☀️'  # Sun

    # Wind conditions
    elif any(word in weather_lower for word in wind_keywords):
        return '💨'  # Wind blowing

    # Fog conditions
    elif any(word in weather_lower for word in fog_keywords):
        return '🌫️'  # Wind blowing

    # Default ( error handling for unknown or undefined weather)
    else:
        return '🌡️'  # Thermometer (default)


In [35]:
# Define create_precipitation_visualisation() here --- get precipitation data through JSON (rain_chance & precip)
def create_precipitation_visualisation(location, day_index=0): #the default is 0 or today
    data = get_weather_data(location, 'json')
    if not data:
        print("Failed to get data.")
        return

  #visualize precipitaion hourly using graphs
    #index of wttr (today 0,tommorrow 1,after tomorrow 2)
    hourly = data['weather'][day_index]['hourly']
    times = [f"{int(h['time'])//100:02d}:00" for h in hourly]
    precip = [float(h['precipMM']) for h in hourly]
    rain_chance = [int(h['chanceofrain']) for h in hourly]

#making graphs
    fig, ax1 = plt.subplots()

    ax1.plot(times, precip, 'b-o', label='Precipitation (mm)')
    ax1.set_xlabel('24 Hour')
    ax1.set_ylabel('Precipitation (mm)', color='b')
    ax1.tick_params(axis='y', labelcolor='b')

    ax2 = ax1.twinx()
    ax2.plot(times, rain_chance, 'g--x', label='Chance of Rain (%)')
    ax2.set_ylabel('Chance of Rain (%)', color='g')
    ax2.tick_params(axis='y', labelcolor='g')

    if day_index == 0:
        days = 'Today'
    elif day_index == 1:
        days = 'Tomorrow'
    else:
        days = 'After-Tomorrow'

#making title for graphs
    plt.title(f'{days} Graphical Representation of Precipitation and Rainfall Probability at {location.capitalize()}')
    plt.grid(True)
    plt.show()

## 🤖 Natural Language Processing

In [36]:
# Define parse_weather_question() and generate_weather_response() here - USING NLP
def parse_weather_question(question, location): #get question & location from user
    # Define regex pattern to detect time - time_pattern is a variable consists of list (when) - regex used to read all time_pattern even the random ones ('r' to saving)
    time_pattern = r"(today|tomorrow|after[-\s]?tomorrow|day[-\s]?after[-\s]?tomorrow)"

    # Lists of outdoor and indoor activities
    outdoor_activities = r"(run|swim|sports|tennis|cycling|hiking|jogging|fishing|camping)"
    indoor_activities = r"(cooking|reading|studying|watching movies|baking|painting|knitting|gaming)"

    # Search for time in the question - ('re' to saving) - use match for lists: time,outdoor,indoor
    time_match = re.search(time_pattern, question, re.IGNORECASE)
    # Search for outdoor activity
    outdoor_match = re.search(outdoor_activities, question, re.IGNORECASE)
    # Search for indoor activity
    indoor_match = re.search(indoor_activities, question, re.IGNORECASE)

    # Default time to "today" if not found
    time = "today"
    activity = None  # Default no activity found
    activity_type = None  # Default no activity type found

    if time_match:
        time = time_match.group(1).lower()  # Extract time (today, tomorrow, after-tomorrow)

    if outdoor_match:
        activity = outdoor_match.group(1).lower()
        activity_type = "Outdoor"
    elif indoor_match:
        activity = indoor_match.group(1).lower()
        activity_type = "Indoor"

    # Calculate target date based on time
    target_date = datetime.now()
    if time == "tomorrow":
        target_date += timedelta(days=1)
    elif time in ["after-tomorrow", "day after tomorrow"]:
        target_date += timedelta(days=2)

    # Call function to display weather based on location and date - to call all function based on (location & target_date)
    if location and activity and activity_type:
        weather = create_temperature_visualisation(location, target_date)
        if weather is None:
            print("Weather data not available.")
            return

        response = generate_weather_response(
            location=weather['location'],
            date_str=weather['date'],
            activity=activity,
            activity_type=activity_type,
            weather_desc=weather['weather_desc'],
            weather_emoji=weather['weather_emoji'],
            min_temp=weather['min_temp'],
            max_temp=weather['max_temp'],
            avg_temp=weather['avg_temp'],
            time = time
        )
        print(response)
    else:
        print("Could not detect location, activity, or time properly.")
    #----------------------------------------------------------------------
def generate_weather_response(location,time, date_str, activity, activity_type, weather_desc, weather_emoji, min_temp, max_temp, avg_temp): #displaying answer function for user's questions
    desc = weather_desc.lower()
    avg_temp = float(avg_temp)

#bad weather keywords lists
    bad_weather_keywords = ['rain', 'thunder', 'fog', 'mist', 'shower', 'snow', 'drizzle', 'downpour', 'blizzard', 'sleet', 'ice', 'hail', 'flurry']

    bad_weather = any(word in desc for word in bad_weather_keywords)

#suggestions response to user's activities
    if activity_type.lower() == "outdoor":
        if bad_weather or avg_temp < 15:
            suggestion = (f"The weather is bad or cold ({avg_temp}°C), so outdoor activities like {activity} are not recommended. Better to stay indoors.")
        elif 15 <= avg_temp <= 30:
            suggestion = (f"The weather and temperature ({avg_temp}°C) are comfortable, perfect for outdoor activities like {activity}.")
        else:
            suggestion = (f"It's very hot ({avg_temp}°C). Even if weather is good, be careful doing outdoor activities like {activity}. Indoor activities are recommended.")
    else:  # indoor
        if bad_weather or avg_temp < 15:
            suggestion = (f"The weather is bad or cold ({avg_temp}°C), so indoor activities like {activity} are the best choice.")
        else:
            suggestion = (f"The weather is good ({avg_temp}°C). Indoor activities like {activity} are fine, but you could also try outdoor activities like jogging or basketball.")

#print the result
    return (
        "\n" + "-" * 53
        +"\n" + " " * 20 + "ACTIVITY SUGGESTION"
        +"\n" + "-" * 53
        +"\n" + f"📍 Location   : {location}"
        +"\n" + f"📅 Date       : {time.capitalize()}, {date_str}"
        +"\n" + f"🏃 Activity   : ({activity_type.capitalize()}) - {activity.capitalize()}"
        +"\n" + f"🌤️ Weather    : {weather_emoji} - {weather_desc}"
        +"\n" + f"🌡️ Temperature: Min {min_temp}°C | Max {max_temp}°C | Avg {avg_temp}°C"
        +"\n"
        +"\n" + "✅ Suggestion:"
        +"\n" + f"{time.capitalize()}, " +suggestion
        +"\n" +"-" * 53 + "\n"
    )


## 🧭 User Interface

In [37]:
# Define menu functions using pyinputplus or ipywidgets here
def main():
  name = input('🤖: Hi! Please enter your name: ')
  location = input(f'🤖: Which location would you like to check, {name}? ')
  while True:
        print(f'🤖 Hello! {name}, Welcome to Weather Wise. How can I help you at {location}?')
        print("-" * 53)
        print('Please enter your choice (1 - 5):')
        print(f'1. Check Current Weather & View Weather Forecast at {location}')
        print(f'2. Check Current Weather Hourly at {location}')
        print(f'3. Activities Suggestions & Weather Prediction assistant at {location}')
        print('4. Check Other Location')
        print('5. Exit Program')

        choice = input('Enter your choice (1-5): ')

        if choice == "1":
           get_weather_data(location)
        elif choice == "2":
            print("Select day for weather visualization:")
            print("1. Today")
            print("2. Tomorrow")
            print("3. Day after tomorrow")
            day_choice = input("Enter choice (1-3): ").strip()
            if day_choice in ['1', '2', '3']:
                day_index = int(day_choice) - 1  # Convert to 0-based index
            else:
                print("Invalid choice, defaulting to Today.")
                day_index = 0
            create_precipitation_visualisation(location, day_index=day_index)
        elif choice == "3":
            question = input("Enter your weather question: ")
            parse_weather_question(question, location)
        elif choice == "4":
            location = input("Enter new location: ")
            print(f'🤖: Your new location is {location}')
        elif choice == "5":
            print(f'Thankyou! Have a nice day {name} 🌸')
            break
        else:
            print("\033[1;31mInvalid choice. Please type 1-5 as a number!\033[0m") #error_handling


## 🧩 Main Application Logic

In [38]:
# Tie everything together here
if __name__ == "__main__":
    title = "Weather Wise"
    blue_bold = "\033[1;94m"  # 1 for bold, 94 for light blue
    color = "\033[0m"

    print("=" * 53)
    print("" + blue_bold + title.center(51) + color + "")
    print("=" * 53)

    main()

                    Weather Wise                   
🤖: Hi! Please enter your name: Zizi
🤖: Which location would you like to check, Zizi? Perth
🤖 Hello! Zizi, Welcome to Weather Wise. How can I help you at Perth?
-----------------------------------------------------
Please enter your choice (1 - 5):
1. Check Current Weather & View Weather Forecast at Perth
2. Check Current Weather Hourly at Perth
3. Activities Suggestions & Weather Prediction assistant at Perth
4. Check Other Location
5. Exit Program
Enter your choice (1-5): 1
Timeout Error: HTTPSConnectionPool(host='wttr.in', port=443): Read timed out. (read timeout=10)
🤖 Hello! Zizi, Welcome to Weather Wise. How can I help you at Perth?
-----------------------------------------------------
Please enter your choice (1 - 5):
1. Check Current Weather & View Weather Forecast at Perth
2. Check Current Weather Hourly at Perth
3. Activities Suggestions & Weather Prediction assistant at Perth
4. Check Other Location
5. Exit Program
Enter your

## 🧪 Testing and Examples

In [ ]:
# Include sample input/output for each function

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.